In [51]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import json



# 1. labeled_data

In [52]:
with open("../data/labeled_data.json", "r") as json_file:
    data = json.load(json_file)

In [53]:
y = np.array([entry['Manual Score'] for entry in data])
X = np.array([np.array(list({k : v for k, v in entry.items() if k != 'Manual Score'}.values())) for entry in data])

print("X", X.shape)
print("y", y.shape)

X (130, 11)
y (130,)


In [54]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=0.2)
print("train set", X_train.shape, y_train.shape)
print("eval set", X_eval.shape, y_eval.shape)

train set (104, 11) (104,)
eval set (26, 11) (26,)


In [55]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_eval)

mse = mean_squared_error(y_eval, y_pred)
r2 = r2_score(y_eval, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 5.4457517068354475
R-squared: 0.03710814139444363


# 2. labeled_data_1

In [56]:
with open("../data/labeled_data_1.json", "r") as json_file:
    data = json.load(json_file)

In [59]:
y = np.array([entry['Manual Score'] for entry in data])
X = np.array([np.array(list({k : v for k, v in entry.items() if k != 'Manual Score'}.values())) for entry in data])

model = LinearRegression()
model.fit(X, y)

y_pred = model.predict(X)

mse = mean_squared_error(y, y_pred)
r2 = r2_score(y, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 4.277371459067167
R-squared: 0.12166359915126801
